<a href="https://colab.research.google.com/github/CP200435/-_TEAM_9124/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A_TEAM_9124.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [2]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.5/112.6 GB disk)


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
#下載資料集
import gdown
import os
import shutil

#下載training_image.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1FfrsOgeJKSHtRM3Q649HJvSF1l4blxpl","/content/training_image.zip")
gdown.download("https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6","/content/training_image.zip")

#下載training_label.zip
# gdown.download("https://drive.google.com/uc?export=download&id=1KmYS2JhAexpWWlzlo_qwoX1TifA1COWW","/content/training_label.zip")
gdown.download("https://drive.google.com/uc?export=download&id=1K4n7G8zVK0_PtgkkAQFBEXJs-zJpElKG","/content/training_label.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6
From (redirected): https://drive.google.com/uc?export=download&id=15jWkS3l4PFqvKAL9DW9DNzl94p4s3P_6&confirm=t&uuid=f4f03d4d-1b77-40e8-9051-b896362773ba
To: /content/training_image.zip
100%|██████████| 1.83G/1.83G [00:21<00:00, 83.7MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1K4n7G8zVK0_PtgkkAQFBEXJs-zJpElKG
To: /content/training_label.zip
100%|██████████| 669k/669k [00:00<00:00, 77.0MB/s]


'/content/training_label.zip'

In [5]:
from textwrap import dedent
from pathlib import Path

data_yaml = dedent(
    """
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve
"""
).strip()

config_path = Path("aortic_valve_colab.yaml")
config_path.write_text(data_yaml + "\n", encoding="utf-8")

print(f"YAML saved to: {config_path.resolve()}")
print(data_yaml)


YAML saved to: /content/aortic_valve_colab.yaml
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve


In [ ]:
#下載資料集
import gdown
import os
!mkdir ./datasets
!mkdir ./datasets/test
gdown.download("https://drive.google.com/uc?export=download&id=13y81TV-z4_fbm7fCmjb0w_FvnPEZIHHa","/content/datasets/testing.zip")
!unzip '/content/datasets/testing' -d '/content/datasets/test/tmp'

In [7]:
import os, zipfile, shutil  #week10hw

def unzip_if_needed(zip_path, dest_dir):
    if os.path.isdir(dest_dir):
        return

    if os.path.exists(zip_path):
        os.makedirs(dest_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(dest_dir)

def find_patient_root(root):
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root

# * 解壓縮 (不使用外部 unzip 指令)
unzip_if_needed("training_image.zip", "../training_image")
unzip_if_needed("training_label.zip", "../training_label")

IMG_ROOT = find_patient_root("../training_image")
LBL_ROOT = find_patient_root("../training_label")

# * 建立輸出資料夾
for p in ["./datasets/train/images", "./datasets/train/labels",
          "./datasets/val/images", "./datasets/val/labels",
          "./datasets/test/images", "./datasets/test/labels"]:
    os.makedirs(p, exist_ok=True)

In [8]:
#week10
def move_patients(start, end, split):
    # train/val 讀前幾筆病患，後面當作 test 保留完整影像
    moved = 0

    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)
        if not os.path.isdir(img_dir):
            continue

        if split == "test":
            label_lookup = {}
            if os.path.isdir(lbl_dir):
                for frame in os.listdir(lbl_dir):
                    if frame.endswith(".txt"):
                        label_lookup[os.path.splitext(frame)[0]] = os.path.join(lbl_dir, frame)

            for frame in os.listdir(img_dir):
                if not frame.lower().endswith(".png"):
                    continue

                base = os.path.splitext(frame)[0]
                img_path = os.path.join(img_dir, frame)
                dst_img = f"./datasets/{split}/images/{base}.png"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                lbl_path = label_lookup.get(base)
                if lbl_path and os.path.exists(lbl_path):
                    dst_lbl = f"./datasets/{split}/labels/{base}.txt"
                    if os.path.exists(dst_lbl):
                        os.remove(dst_lbl)
                    shutil.move(lbl_path, dst_lbl)

                moved += 1

        else:
            if not os.path.isdir(lbl_dir):
                continue

            for lbl_name in os.listdir(lbl_dir):
                if not lbl_name.endswith(".txt"):
                    continue

                base = os.path.splitext(lbl_name)[0]
                img_path = os.path.join(img_dir, base + ".png")
                lbl_path = os.path.join(lbl_dir, lbl_name)
                if not os.path.exists(img_path):
                    continue

                dst_img = f"./datasets/{split}/images/{base}.png"
                dst_lbl = f"./datasets/{split}/labels/{base}.txt"

                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)

                if os.path.exists(dst_lbl):
                    os.remove(dst_lbl)
                shutil.move(lbl_path, dst_lbl)

                moved += 1

    return moved


In [9]:
#week10
n_train = move_patients(1, 40, "train")
n_val = move_patients(41, 45, "val")
n_test = move_patients(46, 50, "test")

print("完成移動: train {} | val {} | test {}".format(n_train, n_val, n_test))

完成移動: train 2168 | val 292 | test 1817


In [10]:
print('訓練集圖片數量 : ',len(os.listdir("./datasets/train/images")))
print('訓練集標記數量 : ',len(os.listdir("./datasets/train/labels")))
print('驗證集圖片數量 : ',len(os.listdir("./datasets/val/images")))
print('驗證集標記數量 : ',len(os.listdir("./datasets/val/labels")))
print('測試集圖片數量 : ',len(os.listdir("./datasets/test/images")))
print('測試集標記數量 : ',len(os.listdir("./datasets/test/labels")))
print('總圖片數量: ',len(os.listdir("./datasets/train/images")) + len(os.listdir("./datasets/val/images")) + len(os.listdir("./datasets/test/images")))
print('總標記數量: ',len(os.listdir("./datasets/train/labels")) + len(os.listdir("./datasets/val/labels")) + len(os.listdir("./datasets/test/labels")))

訓練集圖片數量 :  2168
訓練集標記數量 :  2168
驗證集圖片數量 :  292
驗證集標記數量 :  292
測試集圖片數量 :  1817
測試集標記數量 :  327
總圖片數量:  4277
總標記數量:  2787


In [11]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt') #初次訓練使用YOLO官方的預訓練模型，如要使用自己的模型訓練可以將'yolo12n.pt'替換掉
results = model.train(data="./aortic_valve_colab.yaml",
            epochs=20, #跑幾個epoch
            batch=16, #batch_size
            imgsz=640, #圖片大小640*640
            device=0 #使用GPU進行訓練
            )

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./aortic_valve_colab.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

In [12]:
import os
import shutil

base_root = "/content/datasets/test/tmp"
dst_root1 = "/content/datasets/test/images1"
dst_root2 = "/content/datasets/test/images2"

os.makedirs(dst_root1, exist_ok=True)
os.makedirs(dst_root2, exist_ok=True)

# 自動找到第一個「直屬子資料夾含 patient*」的目錄
patient_root = base_root
for dirpath, dirnames, _ in os.walk(base_root):
    if any(d.lower().startswith("patient") for d in dirnames):
        patient_root = dirpath
        break

# 收集所有圖片路徑（只看直屬的 patient 資料夾）
all_files = []
for patient_folder in os.listdir(patient_root):
    patient_path = os.path.join(patient_root, patient_folder)
    if os.path.isdir(patient_path) and patient_folder.lower().startswith("patient"):
        for fname in os.listdir(patient_path):
            if fname.lower().endswith(".png"):
                all_files.append(os.path.join(patient_path, fname))

# 按名稱排序並對半移動
all_files.sort()
half = len(all_files) // 2

for f in all_files[:half]:
    shutil.move(f, os.path.join(dst_root1, os.path.basename(f)))

for f in all_files[half:]:
    shutil.move(f, os.path.join(dst_root2, os.path.basename(f)))

print(f"來源根目錄：{patient_root}")
print(f"完成移動！總共 {len(all_files)} 張，前半 {half} 張到 images1，後半 {len(all_files)-half} 張到 images2")


來源根目錄：/content/datasets/test/tmp/testing_image
完成移動！總共 16620 張，前半 8310 張到 images1，後半 8310 張到 images2


In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt') #請自行更改最新的best.pt檔路徑
results = model.predict(source="./datasets/test/images1/",
              save=True,
              imgsz=640,
              device=0
              )

In [14]:
print(len(results))

8310


In [15]:
!mkdir ./predict_txt/
output_file = open('./predict_txt/images1.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [16]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')
results = model.predict(source="./datasets/test/images2/",
              save=True,
              imgsz=640,
              device=0
              )

In [18]:
output_file = open('./predict_txt/images2.txt', 'w')
for i in range(len(results)):
    # 取得圖片檔名（不含副檔名）
    filename = results[i].path.split('/')[-1].split('.png')[0]

    # 取得預測框數量
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())

    # 如果有預測框
    if box_num > 0:
        for j in range(box_num):
            # 提取資訊
            label = int(boxes.cls[j].item())  # 類別
            conf = boxes.conf[j].item()       # 信心度
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()  # 邊界框座標

            # 建立一行資料
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)

# 關閉輸出檔案
output_file.close()

In [19]:
file1 = "./predict_txt/images1.txt"
file2 = "./predict_txt/images2.txt"
output = "./predict_txt/merged.txt"

with open(output, "w", encoding="utf-8") as fout:
    for f in [file1, file2]:
        if os.path.exists(f):
            with open(f, "r", encoding="utf-8") as fin:
                fout.writelines(fin.readlines())

print(f"合併完成 -> {output}")


合併完成 -> ./predict_txt/merged.txt


In [20]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

NameError: name 'all_files' is not defined

In [21]:
!yolo detect val model=/content/runs/detect/train/weights/best.pt data=aortic_valve_colab.yaml split=test save_txt save_conf

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 36.6±9.2 MB/s, size: 109.1 KB)
val: Scanning /content/datasets/test/labels... 327 images, 1490 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1817/1817 1.3Kit/s 1.4s
val: New cache created: /content/datasets/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 114/114 5.1it/s 22.3s
                   all       1817        327      0.896      0.893      0.941      0.597
Speed: 1.5ms preprocess, 3.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [22]:
import os
import glob
import cv2
import yaml
import shutil
from pathlib import Path
from ultralytics import YOLO

# ==================== 1. 設定區 (請確認這些路徑) ====================
# 模型路徑 (請確認你的訓練權重路徑是否正確)
MODEL_PATH = 'runs/detect/train/weights/best.pt'
# 資料集設定檔
YAML_PATH = 'aortic_valve_colab.yaml'
# 測試資料圖片路徑 (YOLO驗證時需要的原始圖片)
TEST_IMAGES_DIR = 'datasets/test/images'
# 測試資料標籤路徑 (Ground Truth)
TEST_LABELS_DIR = 'datasets/test/labels'
# 輸出報告的資料夾
OUTPUT_DIR = 'final_report_output'

# ==================== 2. 執行 YOLO 驗證 (關鍵修正) ====================
print("🚀 正在執行 YOLO 驗證，並強制輸出預測 txt 檔...")

# 檢查模型是否存在，不存在則提示
if not os.path.exists(MODEL_PATH):
    print(f"❌ 錯誤：找不到模型權重 {MODEL_PATH}，請檢查路徑。")
else:
    model = YOLO(MODEL_PATH)
    # 這裡的重點是 save_txt=True (存標籤) 和 save_conf=True (存信心度)
    # project 和 name 指定輸出位置，方便我們等一下抓取
    results = model.val(
        data=YAML_PATH,
        split='test',
        save_txt=True,
        save_conf=True,
        conf=0.25,      # 信心度門檻，可自行調整
        iou=0.5,        # NMS IoU 門檻
        project='runs/detect',
        name='val_final', # 固定輸出到 runs/detect/val_final，方便讀取
        exist_ok=True     # 如果資料夾存在就覆蓋，避免生出 val2, val3
    )

print("✅ YOLO 驗證完成！")

# ==================== 3. 驗證結果分析與報告生成 ====================
print("📊 開始進行 IoU 比對與報告生成...")

# 定義預測結果路徑 (因為上面指定了 name='val_final'，所以路徑是固定的)
PRED_DIR = 'runs/detect/val_final/labels'

# 建立輸出目錄
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR) # 清空舊的
os.makedirs(OUTPUT_DIR)

# 讀取類別名稱
class_names = {0: 'object'} # 預設
if hasattr(model, 'names'):
    class_names = model.names

# 準備統計變數
stats = {
    "total_images": 0, "total_gt": 0, "total_pred": 0,
    "tp": 0, "fp": 0, "fn": 0, "dup": 0
}
report_rows = []

# 讀取圖片列表
img_paths = sorted(glob.glob(os.path.join(TEST_IMAGES_DIR, '*')))
if not img_paths:
    print(f"❌ 警告：在 {TEST_IMAGES_DIR} 找不到測試圖片！")

def parse_yolo_txt(path, width, height):
    """讀取 YOLO txt 轉為像素座標 [class, x1, y1, x2, y2, conf]"""
    boxes = []
    if os.path.exists(path):
        with open(path, 'r') as f:
            for line in f:
                d = list(map(float, line.strip().split()))
                cls = int(d[0])
                cx, cy, w, h = d[1], d[2], d[3], d[4]
                conf = d[5] if len(d) > 5 else 1.0

                x1 = int((cx - w/2) * width)
                y1 = int((cy - h/2) * height)
                x2 = int((cx + w/2) * width)
                y2 = int((cy + h/2) * height)
                boxes.append({'cls': cls, 'box': [x1, y1, x2, y2], 'conf': conf})
    return boxes

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# 逐張圖片處理
for img_path in img_paths:
    stats["total_images"] += 1
    filename = os.path.basename(img_path)
    file_id = os.path.splitext(filename)[0]

    # 讀取圖片
    img = cv2.imread(img_path)
    if img is None: continue
    h, w = img.shape[:2]

    # 讀取 GT 和 Pred
    gt_path = os.path.join(TEST_LABELS_DIR, file_id + '.txt')
    pred_path = os.path.join(PRED_DIR, file_id + '.txt')

    gts = parse_yolo_txt(gt_path, w, h)
    preds = parse_yolo_txt(pred_path, w, h)

    stats["total_gt"] += len(gts)
    stats["total_pred"] += len(preds)

    # 匹配邏輯
    gt_matched = [False] * len(gts)
    pred_matched = [False] * len(preds)

    # 1. 找 TP (正確預測)
    for i, p in enumerate(preds):
        best_iou = 0
        best_gt_idx = -1
        for j, g in enumerate(gts):
            if p['cls'] == g['cls']: # 類別需相同
                iou = compute_iou(p['box'], g['box'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j

        if best_iou >= 0.5:
            if not gt_matched[best_gt_idx]:
                gt_matched[best_gt_idx] = True
                pred_matched[i] = True
                stats["tp"] += 1
                # 繪圖: TP (紅色實線)
                cv2.rectangle(img, (p['box'][0], p['box'][1]), (p['box'][2], p['box'][3]), (0, 0, 255), 1)
                cv2.putText(img, f"{p['conf']:.2f}", (p['box'][0], p['box'][1]-5), 0, 0.4, (0, 0, 255), 1)
            else:
                stats["dup"] += 1
                report_rows.append(f"| {filename} | 重複偵測 | IOU: {best_iou:.2f} |")
        else:
            # 這是 FP (如果 IoU 不夠高)
            pass

    # 2. 找 FP (誤報)
    for i, p in enumerate(preds):
        if not pred_matched[i]:
            stats["fp"] += 1
            report_rows.append(f"| {filename} | 誤報 (FP) | Conf: {p['conf']:.2f} |")
            # 繪圖: FP (黃色虛線效果 - 這裡用黃色實線代替)
            cv2.rectangle(img, (p['box'][0], p['box'][1]), (p['box'][2], p['box'][3]), (0, 255, 255), 1)
            cv2.putText(img, "FP", (p['box'][0], p['box'][3]+15), 0, 0.4, (0, 255, 255), 1)

    # 3. 找 FN (漏測)
    for j, g in enumerate(gts):
        if not gt_matched[j]:
            stats["fn"] += 1
            report_rows.append(f"| {filename} | 漏測 (FN) | Class: {g['cls']} |")
            # 繪圖: FN (綠色實線) GT
            cv2.rectangle(img, (g['box'][0], g['box'][1]), (g['box'][2], g['box'][3]), (0, 255, 0), 1)
            cv2.putText(img, "GT(Miss)", (g['box'][0], g['box'][1]-5), 0, 0.4, (0, 255, 0), 1)

    # 儲存圖片
    cv2.imwrite(os.path.join(OUTPUT_DIR, filename), img)

# 生成 Markdown 報告
report_path = '詳細驗證報告.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(f"# YOLO 模型驗證報告\n\n")
    f.write(f"## 1. 總體統計\n")
    f.write(f"- 測試圖片總數: {stats['total_images']}\n")
    f.write(f"- 標註物件數 (GT): {stats['total_gt']}\n")
    f.write(f"- 模型預測數 (Pred): {stats['total_pred']}\n\n")
    f.write(f"### 效能指標\n")
    f.write(f"- **正確預測 (TP)**: {stats['tp']}\n")
    f.write(f"- **漏報 (FN)** (該抓沒抓到): {stats['fn']}\n")
    f.write(f"- **誤報 (FP)** (亂抓): {stats['fp']}\n")
    f.write(f"- **重複偵測**: {stats['dup']}\n\n")
    f.write(f"## 2. 詳細異常列表\n")
    f.write("| 檔名 | 錯誤類型 | 詳情 |\n")
    f.write("|---|---|---|\n")
    for row in report_rows[:100]: # 避免報告太長，只列出前100筆異常
        f.write(row + "\n")
    if len(report_rows) > 100:
        f.write(f"| ... | ... | (還有 {len(report_rows)-100} 筆異常未列出) |\n")

print(f"\n✅ 全部完成！")
print(f"1. 視覺化圖片已存入: {OUTPUT_DIR}/ (綠色=漏測, 紅色=預測, 黃色=誤報)")
print(f"2. 詳細報告已存為: {report_path}")

# 打包下載 (選用)
# shutil.make_archive('驗證結果打包', 'zip', OUTPUT_DIR)
# print("已建立 '驗證結果打包.zip'")

🚀 正在執行 YOLO 驗證，並強制輸出預測 txt 檔...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 540.2±885.6 MB/s, size: 107.1 KB)
val: Scanning /content/datasets/test/labels.cache... 327 images, 1490 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1817/1817 2.8Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 114/114 6.9it/s 16.4s
                   all       1817        327      0.906      0.893       0.94      0.634
Speed: 1.7ms preprocess, 3.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /content/runs/detect/val_final
✅ YOLO 驗證完成！
📊 開始進行 IoU 比對與報告生成...

✅ 全部完成！
1. 視覺化圖片已存入: final_report_output/ (綠色=漏測, 紅色=預測, 黃色=誤報)
2. 詳細報告已存為: 詳細驗證報告.md
